# Website 1 - College Dekho

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import lxml
import requests
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [20]:
#Getting all the States Data
data = pd.read_excel(r"C:\Users\jaske\OneDrive\Desktop\NGO\Data\states.xlsx")
data["states"] = data["states"].str.lower()
diction = dict(zip(data.states, data.number))
print(diction)
diction = {'maharashtra': 2783}
a = []

{'daman-and-diu': 1, 'dubai': 1, 'andaman_nicobar': 3, 'dadra_and_nagar_haveli': 5, 'arunachal-pradesh': 14, 'sikkim': 15, 'mizoram': 22, 'manipur': 23, 'nagaland': 25, 'tripura': 34, 'meghalaya': 40, 'goa': 56, 'chandigarh': 66, 'pondicherry': 82, 'jammu-kashmir': 163, 'himachal-pradesh': 167, 'jharkhand': 211, 'uttaranchal': 254, 'chhattisgarh': 264, 'assam': 295, 'bihar': 397, 'delhi': 456, 'orissa': 477, 'punjab': 670, 'telangana': 699, 'rajasthan': 835, 'haryana': 870, 'madhya-pradesh': 888, 'kerala': 926, 'gujarat': 955, 'west-bengal': 1104, 'andhra-pradesh': 1232, 'karnataka': 1598, 'tamil-nadu': 2033, 'uttar-pradesh': 2368, 'maharashtra': 2783}


In [21]:
#Fuction for Collecting the Links
def linkinfo():    
    body = driver.find_elements_by_class_name("box")
    for b in body:
        title = b.find_elements_by_class_name("title")
        for t in title:
            links = t.find_elements_by_tag_name("a")
            for link in links:
                a.append(link.get_attribute("href"))

In [22]:
#For looping through the different States
for x, y in diction.items():
    state = x
    loop = y//31
    
    url = "https://www.collegedekho.com/colleges-in-{}/"
    chromedriver = r"C:\Users\jaske\Downloads\chromedriver_win32\chromedriver"
    driver = webdriver.Chrome(chromedriver)
    driver.get(url.format(state)) 

    count = 0
    try:
        driver.find_element_by_class_name("loadmorebtn")
    except Exception as e:
        count = 1

    if(count == 0):
        loadingButton = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="loadMoreButton"]')))
        for i in range(1,loop+2):
            driver.execute_script("arguments[0].click();", loadingButton)
            time.sleep(2)
            WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="loadMoreButton"]')))
    
    #Calling the Functions
    linkinfo()
    driver.close()
    print(x)

maharashtra


In [23]:
#Making empty DataFrame
frame = pd.DataFrame(columns = ["name", "courses", "contact", "email", "website", "address","type"])

#Collecting Main Information
for i in a:
    html = requests.get(i).text
    soup = BeautifulSoup(html, "lxml")
    
    #Finding Name
    head = soup.find("div", class_ = "header collegeDetails")
    try:
        typ = head.find("ul", class_="breadcrumb").find("li", class_="bc-item").text
    except Exception as e:
        typ = ""     
    try:
        name = head.find("h1", class_ = "tooltip").text.strip()
        name = name.splitlines()
        name = name[0]
    except Exception as e:
        name = ""
    
    #Finding Other INFO
    body = soup.find("div", class_ = "block collegeContactBlock")
    f = b = c = d = 0
    lists = []
    try:
        main = body. find("div", class_ = "collegeAddress")
        li = main.find_all("li")
        
        for l in li:
            data = l.find_all("div", class_ = "data")
            for da in data:
                lists.append(da.text.strip())
        
        for l in li:
            caption = l.find_all("div", class_ = "label")
            for cap in caption:
                if(cap.text.strip() == "Contact No:"):
                    f = 1
                elif(cap.text.strip() == "Email ID:"):
                    b = 1
                elif(cap.text.strip() == "Website:"):
                    c = 1
                elif(cap.text.strip() == "Address:"):
                    d = 1
        
        if(f == 1):
            contact = lists[0]
            lists.pop(0)
        else:
            contact = ""
        if(b == 1):
            email = lists[0]
            lists.pop(0)
        else:
            email = ""
        if(c == 1):
            website = lists[0]
            lists.pop(0)
        else:
            website = ""
        if(d == 1):
            address = lists[0]
            lists.pop(0)
        else:
            address = ""
    except Exception as e:
        None
    
    
    #Finding Course
    cour = ""
    try:
        courses = soup.find("table", class_ = "couresList")
        course = courses.find_all("td", class_ = "courseName")
        for c in course:
            for i in c.find("a"):
                cour = cour+","+i
    except Exception as e:
        cour = ",None"
    cour = cour[1:]
    
    #Appending information to the DataFrame
    frame = frame.append({"name":name, "courses":cour, "contact":contact, "email":email, "website":website, "address":address, "type":typ}, ignore_index = True)

In [24]:
#Saving to an Excel
frame.to_excel(r"C:\Users\jaske\OneDrive\Desktop\Colleges6.xlsx", index = False)

In [16]:
import pandas as pd
import os

main = pd.DataFrame(columns = ["name","courses","contact","email","website","address","type"])
path = r"C:\Users\jaske\OneDrive\Desktop\folder"
arr = os.listdir(r"C:\Users\jaske\OneDrive\Desktop\folder")
for i in arr:
    df = pd.read_excel(path+"\\"+i)
    main = main.append(df, ignore_index = True)
main = main.drop_duplicates()
main.to_excel(r"C:\Users\jaske\OneDrive\Desktop\final.xlsx")
    

# College 2 - education-india

In [8]:
import pandas as pd
import numpy as np
import lxml
import requests
from bs4 import BeautifulSoup

In [9]:
url = "http://education-india.in/Education/Colleges/?PageNumber={}"
siteurl = "http://education-india.in/Education/Colleges/"
a = []
for i in range(0,180):
    html = requests.get(url.format(i)).text
    soup = BeautifulSoup(html, "lxml")
    main = soup.find("table", class_ = "index")
    links = main.find_all("a")
    for link in links:
        if("CollegeId" in link.attrs["href"]):
            a.append(siteurl + link.attrs["href"])

In [10]:
df = pd.DataFrame(columns=["Name", "Email"])
for i in a:
    html = requests.get(i).text
    soup = BeautifulSoup(html, "lxml")
    main = soup.find("table", class_ = "detail")
    name = main.find("th").text.strip().replace(" Details","")
    links = main.find_all("a")
    for link in links:
        if("mailto" in link.attrs["href"]):
            email = link.attrs["href"].replace("mailto:", "")
    df = df.append({"Name":name, "Email":email}, ignore_index = True)
df.to_excel(r"C:\Users\jaske\OneDrive\Desktop\data3.xlsx")

In [1]:
import pandas as pd
import numpy as np
import lxml
import requests
from bs4 import BeautifulSoup

df = pd.read_excel(r"C:\Users\jaske\OneDrive\Desktop\collegedata2.xlsx")
df1 = df.dropna()
df1.drop_duplicates(keep = False)
df1.to_excel(r"C:\Users\jaske\OneDrive\Desktop\collegedata3.xlsx")